In [1]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import nltk
import emoji
import imaplib
import email
from email.header import decode_header
import pandas as pd
from datetime import datetime, timedelta
import pytz
import configparser
import json
import quopri

In [2]:
config = configparser.ConfigParser()
config.read("app.config")


IMAP_SERVER = config["fetch"]["IMAP_SERVER"]
SMTP_SERVER = config["fetch"]["SMTP_SERVER"]
EMAIL_ACCOUNT = config["fetch"]["EMAIL_ACCOUNT"]
APP_PASSWORD = config["fetch"]["APP_PASSWORD"]

mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, APP_PASSWORD)
mail.select('inbox')

since_date = (datetime.utcnow() - timedelta(hours=24)).strftime('%d-%b-%Y')
result, data = mail.uid('search', None, f'(SINCE {since_date})')

print(f'type of data is {type(data)}')
print(data)

print(f'type of result is {type(result)}')
print(result)

uids = data[0].split()

print(f'type of uids is {type(uids)}')
print(uids)


type of data is <class 'list'>
[b'16494 16495 16496 16497 16498 16499 16500 16501 16502 16503 16504 16505 16506 16507 16508 16509 16510 16511']
type of result is <class 'str'>
OK
type of uids is <class 'list'>
[b'16494', b'16495', b'16496', b'16497', b'16498', b'16499', b'16500', b'16501', b'16502', b'16503', b'16504', b'16505', b'16506', b'16507', b'16508', b'16509', b'16510', b'16511']


In [ ]:
gitdef decode_mime_words(s):
    """
    A function that explicitly handles Quoted-Printable encoding for UTF-8, 
    especially for characters like emojis.
    """
    if not s:
        return ""
    decoded = email.header.decode_header(s)
    decoded_parts = []
    for part, charset in decoded:
        if isinstance(part, bytes):
            if charset and charset.lower() == 'utf-8' and b'=' in part:
                try:
                    decoded_part = quopri.decode(part).decode('utf-8')
                except UnicodeDecodeError:
                    decoded_part = part.decode('utf-8', errors='ignore')
            else:
                decoded_part = part.decode(charset or 'utf-8', errors='ignore')
            decoded_parts.append(decoded_part)
        else:
            decoded_parts.append(str(part)) 

    return ''.join(decoded_parts)



emails = []
cutoff_time = datetime.utcnow().replace(tzinfo=pytz.UTC) - timedelta(hours=24)

for uid in uids:
    result, msg_data = mail.uid('fetch', uid, '(RFC822)')
    if result != 'OK':
        continue

    raw_email = msg_data[0][1]
    msg = email.message_from_bytes(raw_email)

    date_str = msg.get('Date')
    try:
        email_date = email.utils.parsedate_to_datetime(date_str)
        if email_date.tzinfo is None:
            email_date = email_date.replace(tzinfo=pytz.UTC)
    except:
        continue

    if email_date < cutoff_time:
        continue  

    subject = decode_mime_words(msg.get('Subject'))
    sender = decode_mime_words(msg.get('From'))
    recipient = decode_mime_words(msg.get('To'))

   
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain" and not part.get("Content-Disposition"):
                charset = part.get_content_charset()
                body = part.get_payload(decode=True).decode(charset or 'utf-8', errors='ignore')
                break
    else:
        charset = msg.get_content_charset()
        body = msg.get_payload(decode=True).decode(charset or 'utf-8', errors='ignore')

    emails.append({
        'uid': uid.decode(),
        'to': recipient,
        'from': sender,
        'subject': subject,
        'content': body,
        'timestamp': email_date.strftime('%Y-%m-%d %H:%M:%S %Z')
    })

mail.logout()


df = pd.DataFrame(emails)

df.head()


,uid,to,from,subject,content,timestamp
0,16501,yashjoshi30052002@gmail.com,Medium Daily Digest <noreply@medium.com>,How To Wake Up at 5 A.M. Every Day | Bryan Ye ...,Stories for Yash Joshi\r\n@yashjoshi30052002 (...,2025-04-13 01:40:00 UTC
1,16502,yashjoshi30052002@gmail.com,The Hindu <news@newsalertth.thehindu.com>,Editor's Pick | Tamil Nadu govt passes pending...,\r\nEditor's Pick\r\n13 April 2025\r\nIn the E...,2025-04-13 04:39:53 UTC
2,16503,yashjoshi30052002@gmail.com,Prime Video <no-reply@primevideo.com>,Recently added on Prime Video. Watch now!,Find your next favourite on Prime Video\r\n\r\...,2025-04-13 05:03:35 UTC
3,16504,yashjoshi30052002@gmail.com,YONO SBI <YONOSBI@communications.sbi.co.in>,"Single app, simple savings solutions",no text part\r\n\r\n,2025-04-13 11:19:38 UTC+05:30
4,16505,yashjoshi30052002@gmail.com,"""Coursera"" <Coursera@m.learn.coursera.org>",⏰ Ends tomorrow: Get Coursera Plus for ₹8499,"Final chance to unlock 10,000+ programs from G...",2025-04-13 06:12:57 UTC


In [4]:

def clean_content(text):
    text = str(text)  
    text = text.replace("\r\n", " ")  
    text = re.sub(r'\s+', ' ', text) 
    return text.strip() 

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub('', text)


def remove_punc(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
    text = str(text) 
    new_text = []
    for word in text.split():
        if word.lower() in stopwords.words('english'):
            new_text.append(' ')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return ' '.join(x)



df['content'] = df['content'].str.lower()
df['content'] = df['content'].apply(clean_content)
df['content'] = df['content'].apply(remove_url)
df['content'] = df['content'].apply(remove_punc)
for i in range (0 ,df.shape[0]) :
    df['content'][i] = remove_stopwords(df['content'][i])
df['content'] = df['content'].apply(lambda x : emoji.demojize(x))



df['subject'] = df['subject'].str.lower()
df['subject'] = df['subject'].apply(clean_content)
df['subject'] = df['subject'].apply(remove_url)
df['subject'] = df['subject'].apply(remove_punc)
for i in range (0 ,df.shape[0]) :
    df['subject'][i] = remove_stopwords(df['subject'][i])

df['subject'] = df['subject'].apply(lambda x : emoji.demojize(x))




C:\Users\DELL\AppData\Local\Temp\ipykernel_23304\198205819.py:34: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['content'][i] = remove_stopwords(df['content'][i])
C:\Users\DELL\AppData\Local\Temp\ipykernel_23304\198205819.py:34: FutureWar

In [12]:
df.to_pickle("emails.pkl")
df.shape

(11, 6)

In [13]:
df.head(10)

,uid,to,from,subject,content,timestamp
0,16501,yashjoshi30052002@gmail.com,Medium Daily Digest <noreply@medium.com>,wake 5 every day bryan ye better h...,stories yash joshi yashjoshi30052002 ·become...,2025-04-13 01:40:00 UTC
1,16502,yashjoshi30052002@gmail.com,The Hindu <news@newsalertth.thehindu.com>,editors pick tamil nadu govt passes pending bi...,editors pick 13 april 2025 editors pick ne...,2025-04-13 04:39:53 UTC
2,16503,yashjoshi30052002@gmail.com,Prime Video <no-reply@primevideo.com>,recently added prime video watch,find next favourite prime video amazon pri...,2025-04-13 05:03:35 UTC
3,16504,yashjoshi30052002@gmail.com,YONO SBI <YONOSBI@communications.sbi.co.in>,single app simple savings solutions,text part,2025-04-13 11:19:38 UTC+05:30
4,16505,yashjoshi30052002@gmail.com,"""Coursera"" <Coursera@m.learn.coursera.org>",:alarm_clock: ends tomorrow get coursera plus ...,final chance unlock 10000 programs google ...,2025-04-13 06:12:57 UTC
5,16506,yashjoshi30052002@gmail.com,redBus <greetings@travel.e-redbus.in>,yash long weekend coming :backhand_index_poi...,redbus welcome plan bus journey upcoming...,2025-04-13 12:58:18 UTC+05:30
6,16507,yashjoshi30052002@gmail.com,Anthropic <no-reply-5l65bzxcBULVM9UnWStITA@mai...,secure link log claudeai 20250413 170302,doctype htmlhtml langund dirauto xmlns xmlnsvu...,2025-04-13 11:33:02 UTC
7,16508,yashjoshi30052002@gmail.com,huggingface <website@huggingface.co>,hugging face click link confirm email ad...,html head style html fontfamily serif fontsize...,2025-04-13 13:01:43 UTC
8,16509,yashjoshi30052002@gmail.com,huggingface <website@huggingface.co>,hugging face click link confirm email ad...,html head style html fontfamily serif fontsize...,2025-04-13 13:01:49 UTC
9,16510,yashjoshi30052002@gmail.com,Anthropic Team <team@email.anthropic.com>,you’re in—meet claude new ai assistant,welcome claude ai assistant designed hel...,2025-04-13 13:14:59 UTC
